# 加载数据

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 

df = pd.read_csv('./data/df.csv',sep=',')
df  = df.sort_values(by='time',ascending=True).reset_index(drop=True)

BATCHSIZE = 512
LOOKBACK = 5


date = ["2024-02-19",
"2024-03-15",
"2024-04-19",
"2024-05-17",
"2024-06-21",
"2024-07-19",
"2024-08-16",
"2024-09-20",
"2024-10-18",
"2024-11-15",
"2024-12-20",
"2025-01-17"]
df.head()

df['time'] = pd.to_datetime(df['time'])
df['date'] = df['time'].dt.date
df['date'] = df['date'].astype(str)
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute 
# for i in range(1,5):
#     df[f'spread_shift_{i}'] = df.groupby(['hour','minute'])['spread'].shift(i)

df['Expiration_Date'] = df['date'].apply(lambda x: 1 if x in date else 0)
# df.fillna(0,inplace=True)


df.dropna(inplace=True)

df.head()

In [31]:
# !pip install yfinance
# !pip install pandas-datareader

# Import yfinance to make Yahoo Finance API call 
# import yfinance as yf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline
import seaborn as sns
from datetime import datetime
from functools import reduce 

# Import data reader to directly convert Yahoo Finance data into dataframe
from pandas_datareader import data as pdr

# !pip install arch
from arch import arch_model

# ACF plot of standardized residuals
from statsmodels.graphics.tsaplots import plot_acf

# Ljunb-Box test of standardized residuals
from statsmodels.stats.diagnostic import acorr_ljungbox

from sklearn.metrics import mean_absolute_error, mean_squared_error

# Set seaborn plot style
sns.set_style("darkgrid")

# Set matplotlib plot style
style.use("fivethirtyeight")

In [ ]:
df.iloc[100]['time']

In [ ]:
df.info()

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(12,6))
ax.plot(df['time'],df['spread'], color='turquoise')
ax.set(title='MRNA', ylabel='Price per Share') 

# ax.axvline(pd.to_datetime('2020-11-30'), color='slategray', lw=1.2, linestyle='--')
# ax.text(pd.to_datetime('2020-04-12'), max(mrna['Adj Close']+5), 'Vaccine FDA EUA application', color='slategray')
plt.show()

In [ ]:
df['return'] = df['spread'].pct_change().dropna() * 100

# Plot
fig, ax = plt.subplots(figsize=(16,8))
ax.plot(df['time'],df['return'], color='lightcoral')
ax.set(title='MRNA', ylabel='% Return')
plt.show()

In [ ]:
# Make sure we drop nans
df.dropna(inplace=True)

daily_volatility = df[(df['return']<df['return'].max() )&(df['return']>df['return'].min())]['return'].std()
print('Daily volatility: ', '{:.2f}%'.format(daily_volatility))

monthly_trade_days = 21
monthly_volatility = np.sqrt(monthly_trade_days) * daily_volatility
print('Monthly volatility: ', '{:.2f}%'.format(monthly_volatility))

yearly_trade_days = 252
yearly_volatility = np.sqrt(yearly_trade_days) * daily_volatility
print('Yearly volatility: ', '{:.2f}%'.format(yearly_volatility))

In [ ]:
df 

# base_model constant_mean, Normal distribution

In [ ]:
split_size = int(len(df) * 0.8)
df_predict = pd.DataFrame()
for i in range(split_size,len(df)-5):
    df_train = df.iloc[i-500:i]
    df_test = df.iloc[i:i+5]

    basic_gm = arch_model(df_train['spread'], p=1, q=1, mean='constant', vol='GARCH', dist='normal') 
    gm_result = basic_gm.fit()
    gm_forecast = gm_result.forecast(horizon = 5)
    df_predict_tmp = gm_forecast.mean
    for ii in range(5):
        df_predict_tmp[f't.{ii+1}'] = df_test['spread'].values[ii]
    df_predict_tmp['split_index'] = i 
    df_predict_tmp['time'] = df.iloc[i]['time']
    df_predict = pd.concat([df_predict,df_predict_tmp])

df_predict
df_predict.to_csv('./data/df_predict_gmgarch.csv',sep='\t',index=False)


In [ ]:
df_predict
np.sqrt(np.mean((df_predict['h.1']-df_predict['t.1'])**2))
# (df_predict['h.1']-df_predict['t.1'])**2

In [ ]:
gm_result.summary()

In [ ]:
gm_result.plot()
plt.show()

In [ ]:
# Make 5-period ahead forecast
gm_forecast = gm_result.forecast(horizon = 5)

# Print the forecast variance
print(gm_forecast.variance[-1:])

In [ ]:
dir(gm_forecast)

In [ ]:
df_predict = gm_forecast.mean
df_predict['t.1'] = 1
df_predict 


In [ ]:
df.tail()

In [ ]:
# Calculate standardized residual
gm_std_resid = gm_result.resid / gm_result.conditional_volatility

# Plot
fig, ax = plt.subplots(figsize=(8,5))
ax.hist(gm_std_resid, color='salmon', bins=40)
ax.set(title='Distribution of Standardized Residuals')
plt.show()

## 4b. Constant mean, skewed t distribution

In [ ]:
# skewt_gm = arch_model(df['spread'], p=1, q=1, mean='constant', vol='GARCH', dist='skewt') 
# skewt_result = skewt_gm.fit()


split_size = int(len(df) * 0.8)
df_predict = pd.DataFrame()
for i in range(split_size,len(df)-5):
    df_train = df.iloc[i-500:i]
    df_test = df.iloc[i:i+5]

    # basic_gm = arch_model(df_train['spread'], p=1, q=1, mean='constant', vol='GARCH', dist='normal') 
    skewt_gm = arch_model(df_train['spread'], p=1, q=1, mean='constant', vol='GARCH', dist='skewt') 
    skewt_result = skewt_gm.fit()
    skewt_forecast = skewt_result.forecast(horizon = 5)
    df_predict_tmp = skewt_forecast.mean
    for ii in range(5):
        df_predict_tmp[f't.{ii+1}'] = df_test['spread'].values[ii]
    df_predict_tmp['split_index'] = i 
    df_predict_tmp['time'] = df.iloc[i]['time']
    df_predict = pd.concat([df_predict,df_predict_tmp])

df_predict
df_predict.to_csv('./data/df_predict_skewt.csv',sep='\t',index=False)




In [ ]:
skewt_result.summary()

In [ ]:
# Get model estimated volatility
normal_volatility = gm_result.conditional_volatility
skewt_volatility = skewt_result.conditional_volatility

# Plot model fitting results
plt.figure(figsize=(12,6))
plt.plot(skewt_volatility, color = 'gold', label = 'Skewed-t Volatility')
plt.plot(normal_volatility, color = 'turquoise', label = 'Normal Volatility')
plt.plot(df['spread'], color = 'grey', label = 'Daily Returns', alpha = 0.4)
plt.legend(loc = 'upper right', frameon=False)
plt.show()

In [30]:
### 4c. Autoregressive (AR) Mean, skewed t distribution

In [ ]:
# armean_gm = arch_model(df['spread'], p=1, q=1, mean='AR', lags=1, vol='GARCH', dist='skewt') 
# armean_result = armean_gm.fit()


for p in [1,2,3]:
    for q in [1,2,3]:
        split_size = int(len(df) * 0.8)
        df_predict = pd.DataFrame()
        for i in range(split_size,len(df)-5):
            df_train = df.iloc[i-500:i]
            df_test = df.iloc[i:i+5]

            # basic_gm = arch_model(df_train['spread'], p=1, q=1, mean='constant', vol='GARCH', dist='normal') 
            # skewt_gm = arch_model(df_train['spread'], p=1, q=1, mean='constant', vol='GARCH', dist='skewt') 
            armean_gm = arch_model(df_train['spread'], p=p, q=q, mean='AR', lags=1, vol='GARCH', dist='skewt') 
            armean_result = armean_gm.fit()
            armean_forecast = armean_result.forecast(horizon = 5)
            df_predict_tmp = armean_forecast.mean
            for ii in range(5):
                df_predict_tmp[f't.{ii+1}'] = df_test['spread'].values[ii]
            df_predict_tmp['split_index'] = i 
            df_predict_tmp['time'] = df.iloc[i]['time']
            df_predict = pd.concat([df_predict,df_predict_tmp])

        df_predict
        df_predict.to_csv(f'./data/df_predict_armean_{p}_{q}.csv',sep='\t',index=False)

In [ ]:
armean_result.summary()

In [ ]:
armean_volatility = armean_result.conditional_volatility.dropna() # uses lag, first row will be NaN
skewt_volatility = skewt_volatility.iloc[1:] # drop first row

# Plot model fitting results
plt.figure(figsize=(12,6))
plt.plot(skewt_volatility, color = 'gold', label = 'Constant Mean Volatility')
plt.plot(armean_volatility, color = 'turquoise', label = 'AR Mean Volatility')
plt.plot(df['spread'], color = 'grey', label = 'Daily Returns', alpha = 0.4)
plt.legend(loc = 'upper right', frameon=False)
plt.show()



In [ ]:
# Correlation 
np.corrcoef(skewt_volatility, armean_volatility)[0,1]

In [35]:
## 4d. EGARCH (asymmetric shock) 

In [ ]:
# egarch_gm = arch_model(df['spread'], p=1, q=1, o=1, mean='constant', vol='EGARCH', dist='skewt') 
# egarch_result = egarch_gm.fit()


split_size = int(len(df) * 0.8)
df_predict = pd.DataFrame()
for i in range(split_size,len(df)-5):
    df_train = df.iloc[i-500:i]
    df_test = df.iloc[i:i+5]

    # basic_gm = arch_model(df_train['spread'], p=1, q=1, mean='constant', vol='GARCH', dist='normal') 
    # skewt_gm = arch_model(df_train['spread'], p=1, q=1, mean='constant', vol='GARCH', dist='skewt') 
    # armean_gm = arch_model(df_train['spread'], p=1, q=1, mean='AR', lags=1, vol='GARCH', dist='skewt') 
    egarch_gm = arch_model(df_train['spread'], p=1, q=1, o=1, mean='constant', vol='EGARCH', dist='skewt') 
    egarch_result = egarch_gm.fit()
    egarch_forecast = egarch_result.forecast(horizon = 1)
    df_predict_tmp = egarch_forecast.mean
    for ii in range(1):
        df_predict_tmp[f't.{ii+1}'] = df_test['spread'].values[ii]
    df_predict_tmp['split_index'] = i 
    df_predict_tmp['time'] = df.iloc[i]['time']
    df_predict = pd.concat([df_predict,df_predict_tmp])

df_predict
df_predict.to_csv('./data/df_predict_egarch.csv',sep='\t',index=False)



In [ ]:
egarch_result.summary()

In [38]:
## 4e. GJR-GARCH

In [ ]:
gjrgarch_gm = arch_model(df['spread'], p=1, q=1, o=1, mean='constant', vol='GARCH', dist='skewt') 
gjrgarch_result = gjrgarch_gm.fit()



split_size = int(len(df) * 0.8)
df_predict = pd.DataFrame()
for i in range(split_size,len(df)-5):
    df_train = df.iloc[i-500:i]
    df_test = df.iloc[i:i+5]

    # basic_gm = arch_model(df_train['spread'], p=1, q=1, mean='constant', vol='GARCH', dist='normal') 
    # skewt_gm = arch_model(df_train['spread'], p=1, q=1, mean='constant', vol='GARCH', dist='skewt') 
    # armean_gm = arch_model(df_train['spread'], p=1, q=1, mean='AR', lags=1, vol='GARCH', dist='skewt') 
    # egarch_gm = arch_model(df_train['spread'], p=1, q=1, o=1, mean='constant', vol='EGARCH', dist='skewt') 
    gjrgarch_gm = arch_model(df_train['spread'], p=1, q=1, o=1, mean='constant', vol='GARCH', dist='skewt') 
    gjrgarch_result = gjrgarch_gm.fit()
    gjrgarch_forecast = gjrgarch_result.forecast(horizon = 5)
    df_predict_tmp = gjrgarch_forecast.mean
    for ii in range(5):
        df_predict_tmp[f't.{ii+1}'] = df_test['spread'].values[ii]
    df_predict_tmp['split_index'] = i 
    df_predict_tmp['time'] = df.iloc[i]['time']
    df_predict = pd.concat([df_predict,df_predict_tmp])

df_predict.to_csv('./data/df_predict_gjrgarch.csv',sep='\t',index=False)



In [ ]:
gjrgarch_result.summary()

In [ ]:
gjrgarch_result.plot()

In [ ]:
# Function to create dataframe of log likelihood, AIC, BIC for each model
def goodness_of_fit():
  global df
  model_names = ['normal', 'skewt', 'GJR-GARCH', 'EGARCH']
  models = [gm_result, skewt_result, gjrgarch_result, egarch_result]
  likelihood = [model.loglikelihood for model in models]
  aic = [model.aic for model in models]
  bic = [model.bic for model in models]
  dict = {'model':model_names, 'log likelihood':likelihood, 'aic':aic,'bic':bic}
  df = pd.DataFrame(dict).set_index('model')
  return df

goodness_of_fit()

In [ ]:
# Highlight max log likelihood
df.style.highlight_max(subset='log likelihood', color = 'yellow', axis = 0) 

In [ ]:
# Highlight min AIC, BIC
df.style.highlight_min(subset=['aic', 'bic'], color = 'yellow', axis = 0) 

In [ ]:
# Get parameter stats from model summary
parameters = pd.DataFrame({'parameter': gjrgarch_result.params,
                           'p-value': gjrgarch_result.pvalues})

parameters

In [ ]:
# Calculate standardized residual
gjrgarch_std_resid = gjrgarch_result.resid / gjrgarch_result.conditional_volatility

# Plot
fig, ax = plt.subplots(figsize=(8,5))
ax.plot(gjrgarch_std_resid, color='lightcoral')
ax.set(title='Standardized Resduals') 
plt.show()

In [ ]:
import os 
for root, dirs, files in os.walk('./data'):
    for file in files:
        if file.startswith('df_predict'):
            df = pd.read_csv(os.path.join(root, file),sep='\t')
            print(file,np.sqrt(np.mean((df['h.1']-df['t.1'])**2)))


In [ ]:
from metrics import *
import pandas as pd 
metric = Metrics()
import os 
for root, dirs, files in os.walk('./data'):
    for file in files:
        if file.startswith('df_predict'):
            df = pd.read_csv(os.path.join(root, file),sep='\t')
            print(file, metric.MetricsAll(df['t.1'],df['h.1']))
            # print(file,np.sqrt(np.mean((df['h.1']-df['t.1'])**2)))

In [ ]:
from metrics import *
import pandas as pd 
metric = Metrics()
import os 
for root, dirs, files in os.walk('./data'):
    for file in files:
        if file.startswith('lstm'):
            df = pd.read_csv(os.path.join(root, file),sep=',')
            df = df[df['true']!='true']
            df['predict'] = df['predict'].astype(float)
            df['true'] = df['true'].astype(float)
            print(file, metric.MetricsAll(df['true'],df['predict']))
            # print(file,np.sqrt(np.mean((df['h.1']-df['t.1'])**2)))

In [ ]:
from metrics import *
import pandas as pd 
metric = Metrics()
import os 
for root, dirs, files in os.walk('./data'):
    for file in files:
        if file.startswith('fcnn'):
            df = pd.read_csv(os.path.join(root, file),sep=',')
            df = df[df['true']!='true']
            df['predict'] = df['predict'].astype(float)
            df['true'] = df['true'].astype(float)
            print(file, metric.MetricsAll(df['true'],df['predict']))

In [4]:
# (self.mae(y_true,y_pred), self.rmse(y_true,y_pred),self.wmape(y_true,y_pred),self.r_squared(y_true,y_pred))

In [ ]:
from metrics import *
import pandas as pd 
metric = Metrics()
import os 
for root, dirs, files in os.walk('./data'):
    for file in files:
        if file.startswith('cnn'):
            df = pd.read_csv(os.path.join(root, file),sep=',')
            df = df[df['true']!='true']
            df['predict'] = df['predict'].astype(float)
            df['true'] = df['true'].astype(float)
            print(file, metric.MetricsAll(df['true'],df['predict']))

In [ ]:
from metrics import *
import pandas as pd 
metric = Metrics()
import os 
for root, dirs, files in os.walk('./result'):
    for file in files:
        
        df = pd.read_csv(os.path.join(root, file),sep='\t')
        # print(df.head())
        df = df[df['true']!='true']
        df['predict'] = df['predict'].astype(float)
        df['true'] = df['true'].astype(float)
        print(file, metric.MetricsAll(df['true'],df['predict']))

In [10]:


# 过滤garch模型到期日前两天的的结果

In [ ]:
df_1 = pd.read_csv('./data/df_predict_armean.csv',sep='\t')
df_1 

In [ ]:
df_2 = pd.read_csv('./result/df_trendawarelstm_到期日前两天.csv',sep='\t')
df_2 


In [ ]:
df_1[df_1['split_index'].isin(df_2['split_index'].values + 100)] 

In [ ]:

from metrics import *
import pandas as pd 
metric = Metrics()
import os 
for root, dirs, files in os.walk('./data'):
    for file in files:
        if file.startswith('df_predict'):
            df = pd.read_csv(os.path.join(root, file),sep='\t')
            # print(file, metric.MetricsAll(df['t.1'],df['h.1']))
            # print(file,np.sqrt(np.mean((df['h.1']-df['t.1'])**2)))

            print(file, metric.MetricsAll(df[df['split_index'].isin(df_2['split_index'].values + 100)] ['t.1'],
                                        df[df['split_index'].isin(df_2['split_index'].values + 100)] ['h.1']))
            df[df['split_index'].isin(df_2['split_index'].values + 100)].to_csv(f'./result/{file}_到期日前两天.csv',sep='\t')


In [ ]:
df[df['split_index'].isin(df_2['split_index'].values + 100)] 